In [0]:
from delta.tables import *

DeltaTable.create(spark) \
    .tableName("emp_demo") \
        .addColumn("emp_id","INT") \
            .addColumn("emp_name","STRING") \
                .addColumn("gender","STRING") \
                    .addColumn("salary","INT") \
                        .addColumn("dept","STRING") \
                            .property("description","table created for demo purpose") \
                                .location("/FileStore/tables/delta/createtable") \
                                    .execute()


Out[1]: <delta.tables.DeltaTable at 0x7f0698284c10>

In [0]:
%sql

select * from emp_demo

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:
employee_data=[(100,"Stephen","M",2000,"IT"),
               (200,"Philips","M",8000,"HR"),
               (300,"Lara","F",6000,"SALES")]

employee_schema=["emp_id","emp_name","gender","salary","dept"]

df=spark.createDataFrame(data=employee_data,schema=employee_schema)

display(df)

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
200,Philips,M,8000,HR
300,Lara,F,6000,SALES


In [0]:
%sql

select * from emp_demo1

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-508200138522915>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-508200138522915>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("c2VsZWN0ICogZnJvbSBlbXBfZGVtbzE=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/

In [0]:
di=DeltaTable.forPath(spark,"/FileStore/tables/delta/createtable")

In [0]:
%sql

select * from emp_demo

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:
display(di.toDF())

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:
%sql
insert into emp_demo values(100,"Stephen","M",2000,"IT")

num_affected_rows,num_inserted_rows
1,1


In [0]:
# display(di.toDF())
display(spark.sql("select * from emp_demo"))

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
100,Stephen,M,2000,IT
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:
from pyspark.sql.types import *
emp_data=[(200,"Philips","M",8000,"HR")]

emp_schema=StructType([ \
    StructField("emp_id",IntegerType(),False), \
    StructField("emp_name",StringType(),True), \
    StructField("gender",StringType(),True), \
    StructField("salary",IntegerType(),True), \
    StructField("dept",StringType(),True), \
    ])

df=spark.createDataFrame(data=emp_data,schema=emp_schema)

display(df)

emp_id,emp_name,gender,salary,dept
200,Philips,M,8000,HR


In [0]:
%sql

select * from emp_demo

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
100,Stephen,M,2000,IT
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:
df.write.format("delta").mode("append").saveAsTable("emp_demo")

In [0]:
%sql

select * from emp_demo

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
100,Stephen,M,2000,IT
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:
from pyspark.sql.types import *
emp_data=[(300,"Lara","F",6000,"SALES")]

emp_schema=StructType([ \
    StructField("emp_id",IntegerType(),False), \
    StructField("emp_name",StringType(),True), \
    StructField("gender",StringType(),True), \
    StructField("salary",IntegerType(),True), \
    StructField("dept",StringType(),True), \
    ])

df1=spark.createDataFrame(data=emp_data,schema=emp_schema)

display(df1)

emp_id,emp_name,gender,salary,dept
300,Lara,F,6000,SALES


In [0]:
df1.write.insertInto('emp_demo',overwrite=False)

In [0]:
%sql

select * from emp_demo

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:
df1.createOrReplaceTempView("deltadata")

In [0]:
%sql
insert into emp_demo
select * from deltadata

num_affected_rows,num_inserted_rows
1,1


In [0]:
spark.sql("insert into emp_demo select * from deltadata")

Out[20]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
display(spark.sql("select * from emp_demo"))

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:
%sql
delete from emp_demo where emp_id=200

num_affected_rows
4


In [0]:
display(spark.sql("select * from emp_demo"))

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES


In [0]:
spark.sql("delete from emp_demo where emp_id=100")

Out[24]: DataFrame[num_affected_rows: bigint]

In [0]:
display(spark.sql("select * from emp_demo"))

emp_id,emp_name,gender,salary,dept
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES


In [0]:
display(di.toDF())

emp_id,emp_name,gender,salary,dept
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES


In [0]:
di.delete("emp_id=100")

In [0]:
display(di.toDF())

emp_id,emp_name,gender,salary,dept
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES


In [0]:
di.update(
    condition="emp_name='Lara'",
    set={"salary":"15000"}
)

In [0]:
display(di.toDF())

emp_id,emp_name,gender,salary,dept
300,Lara,F,15000,SALES
300,Lara,F,15000,SALES
300,Lara,F,15000,SALES
300,Lara,F,15000,SALES
300,Lara,F,15000,SALES


In [0]:
%sql
desc history emp_demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
23,2025-01-21T05:03:01.000+0000,6367521402730934,molabantisravanth@gmail.com,UPDATE,"Map(predicate -> [""(emp_name#962 = Lara)""])",null,List(508200138522909),0121-045452-7f69q673,22,WriteSerializable,false,"Map(numRemovedFiles -> 5, numRemovedBytes -> 7605, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2630, scanTimeMs -> 1180, numAddedFiles -> 5, numUpdatedRows -> 5, numAddedBytes -> 7605, rewriteTimeMs -> 1449)",null,Databricks-Runtime/12.2.x-scala2.12
22,2025-01-21T05:02:53.000+0000,6367521402730934,molabantisravanth@gmail.com,DELETE,"Map(predicate -> [""(emp_id#961 = 100)""])",null,List(508200138522909),0121-045452-7f69q673,21,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 209, numDeletedRows -> 0, scanTimeMs -> 208, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/12.2.x-scala2.12
21,2025-01-21T05:02:46.000+0000,6367521402730934,molabantisravanth@gmail.com,DELETE,"Map(predicate -> [""(emp_id#6511 = 100)""])",null,List(508200138522909),0121-045452-7f69q673,20,WriteSerializable,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 4563, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2075, numDeletedRows -> 3, scanTimeMs -> 1038, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1036)",null,Databricks-Runtime/12.2.x-scala2.12
20,2025-01-21T05:02:36.000+0000,6367521402730934,molabantisravanth@gmail.com,DELETE,"Map(predicate -> [""(emp_id#5576 = 200)""])",null,List(508200138522909),0121-045452-7f69q673,19,WriteSerializable,false,"Map(numRemovedFiles -> 4, numRemovedBytes -> 6080, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3758, numDeletedRows -> 4, scanTimeMs -> 2386, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1330)",null,Databricks-Runtime/12.2.x-scala2.12
19,2025-01-21T05:02:23.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,18,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
18,2025-01-21T05:02:19.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,17,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
17,2025-01-21T05:02:09.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,16,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
16,2025-01-21T05:01:58.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,15,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1520)",null,Databricks-Runtime/12.2.x-scala2.12
15,2025-01-21T05:01:41.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,14,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
14,2025-01-20T09:17:15.000+0000,6367521402730934,molabantisravanth@gmail.com,RESTORE,"Map(version -> 7, timestamp -> null)",null,List(508200138522909),0120-085650-8syifd7w

In [0]:
%sql

select * from emp_demo version as of 7

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:
%sql

select * from emp_demo

emp_id,emp_name,gender,salary,dept
300,Lara,F,15000,SALES
300,Lara,F,15000,SALES
300,Lara,F,15000,SALES
300,Lara,F,15000,SALES
300,Lara,F,15000,SALES


In [0]:
%sql

restore table emp_demo to version as of 7

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
10644,7,5,7,7605,10644


In [0]:
display(di.toDF())

emp_id,emp_name,gender,salary,dept
100,Stephen,M,2000,IT
100,Stephen,M,2000,IT
300,Lara,F,6000,SALES
300,Lara,F,6000,SALES
200,Philips,M,8000,HR
200,Philips,M,8000,HR
200,Philips,M,8000,HR


In [0]:

fullHistoryDF = di.history()    # get the full history of the table
display(fullHistoryDF)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
24,2025-01-21T05:03:43.000+0000,6367521402730934,molabantisravanth@gmail.com,RESTORE,"Map(version -> 7, timestamp -> null)",null,List(508200138522909),0121-045452-7f69q673,23,Serializable,false,"Map(numRestoredFiles -> 7, removedFilesSize -> 7605, numRemovedFiles -> 5, restoredFilesSize -> 10644, numOfFilesAfterRestore -> 7, tableSizeAfterRestore -> 10644)",null,Databricks-Runtime/12.2.x-scala2.12
23,2025-01-21T05:03:01.000+0000,6367521402730934,molabantisravanth@gmail.com,UPDATE,"Map(predicate -> [""(emp_name#962 = Lara)""])",null,List(508200138522909),0121-045452-7f69q673,22,WriteSerializable,false,"Map(numRemovedFiles -> 5, numRemovedBytes -> 7605, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2630, scanTimeMs -> 1180, numAddedFiles -> 5, numUpdatedRows -> 5, numAddedBytes -> 7605, rewriteTimeMs -> 1449)",null,Databricks-Runtime/12.2.x-scala2.12
22,2025-01-21T05:02:53.000+0000,6367521402730934,molabantisravanth@gmail.com,DELETE,"Map(predicate -> [""(emp_id#961 = 100)""])",null,List(508200138522909),0121-045452-7f69q673,21,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 209, numDeletedRows -> 0, scanTimeMs -> 208, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/12.2.x-scala2.12
21,2025-01-21T05:02:46.000+0000,6367521402730934,molabantisravanth@gmail.com,DELETE,"Map(predicate -> [""(emp_id#6511 = 100)""])",null,List(508200138522909),0121-045452-7f69q673,20,WriteSerializable,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 4563, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2075, numDeletedRows -> 3, scanTimeMs -> 1038, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1036)",null,Databricks-Runtime/12.2.x-scala2.12
20,2025-01-21T05:02:36.000+0000,6367521402730934,molabantisravanth@gmail.com,DELETE,"Map(predicate -> [""(emp_id#5576 = 200)""])",null,List(508200138522909),0121-045452-7f69q673,19,WriteSerializable,false,"Map(numRemovedFiles -> 4, numRemovedBytes -> 6080, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3758, numDeletedRows -> 4, scanTimeMs -> 2386, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1330)",null,Databricks-Runtime/12.2.x-scala2.12
19,2025-01-21T05:02:23.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,18,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
18,2025-01-21T05:02:19.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,17,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
17,2025-01-21T05:02:09.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,16,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
16,2025-01-21T05:01:58.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,15,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1520)",null,Databricks-Runtime/12.2.x-scala2.12
15,2025-01-21T05:01:41.000+0000,6367521402730934,molabantisravanth@gmail.c

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

schema=StructType([
    StructField("emp_id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("city",StringType(),True),
    StructField("country",StringType(),True),
    StructField("contact_no",IntegerType(),True),
])

In [0]:
data=[(1000,"Michael","Columbus","USA",689546323)]
df2=spark.createDataFrame(data=data,schema=schema)
display(df2)

emp_id,name,city,country,contact_no
1000,Michael,Columbus,USA,689546323


In [0]:
%sql

CREATE OR REPLACE TABLE dim_emp(
  emp_id INT,
  name STRING,
  city STRING,
  country STRING,
  contact_no INTEGER
)
USING DELTA
LOCATION "/FileStore/tables/delta_merge"

In [0]:
%sql

select * from dim_emp

emp_id,name,city,country,contact_no


In [0]:
df2.createOrReplaceTempView("source_view")

In [0]:
%sql

select * from source_view

emp_id,name,city,country,contact_no
1000,Michael,Columbus,USA,689546323


In [0]:
%sql
select * from dim_emp

emp_id,name,city,country,contact_no


In [0]:
%sql

MERGE INTO dim_emp as target
USING source_view as source
ON target.emp_id=source.emp_id
WHEN MATCHED
THEN UPDATE SET
target.name=source.name,
target.city=source.city,
target.country=source.country,
target.contact_no=source.contact_no
WHEN NOT MATCHED THEN
INSERT (emp_id,name,city,country,contact_no) VALUES (emp_id,name,city,country,contact_no)


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1


In [0]:
%sql

select * from dim_emp

emp_id,name,city,country,contact_no
1000,Michael,Columbus,USA,689546323


In [0]:
data=[(1000,"Michael","Chicago","USA",689546323),(2000,"Nancy","New york","USA",765234098)]

df=spark.createDataFrame(data=data,schema=schema)

display(df)

emp_id,name,city,country,contact_no
1000,Michael,Chicago,USA,689546323
2000,Nancy,New york,USA,765234098


In [0]:
df.createOrReplaceTempView("source_view")

In [0]:
%sql

select * from source_view

emp_id,name,city,country,contact_no
1000,Michael,Chicago,USA,689546323
2000,Nancy,New york,USA,765234098


In [0]:
%sql

select * from dim_emp

emp_id,name,city,country,contact_no
1000,Michael,Columbus,USA,689546323


In [0]:
%sql

MERGE INTO dim_emp as target
USING source_view as source
ON target.emp_id=source.emp_id
WHEN MATCHED
THEN UPDATE SET
target.name=source.name,
target.city=source.city,
target.country=source.country,
target.contact_no=source.contact_no
WHEN NOT MATCHED THEN
INSERT (emp_id,name,city,country,contact_no) VALUES (emp_id,name,city,country,contact_no)


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,1,0,1


In [0]:
%sql

select * from dim_emp

emp_id,name,city,country,contact_no
1000,Michael,Chicago,USA,689546323
2000,Nancy,New york,USA,765234098


In [0]:
%sql

desc history emp_demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
24,2025-01-21T05:03:43.000+0000,6367521402730934,molabantisravanth@gmail.com,RESTORE,"Map(version -> 7, timestamp -> null)",null,List(508200138522909),0121-045452-7f69q673,23,Serializable,false,"Map(numRestoredFiles -> 7, removedFilesSize -> 7605, numRemovedFiles -> 5, restoredFilesSize -> 10644, numOfFilesAfterRestore -> 7, tableSizeAfterRestore -> 10644)",null,Databricks-Runtime/12.2.x-scala2.12
23,2025-01-21T05:03:01.000+0000,6367521402730934,molabantisravanth@gmail.com,UPDATE,"Map(predicate -> [""(emp_name#962 = Lara)""])",null,List(508200138522909),0121-045452-7f69q673,22,WriteSerializable,false,"Map(numRemovedFiles -> 5, numRemovedBytes -> 7605, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2630, scanTimeMs -> 1180, numAddedFiles -> 5, numUpdatedRows -> 5, numAddedBytes -> 7605, rewriteTimeMs -> 1449)",null,Databricks-Runtime/12.2.x-scala2.12
22,2025-01-21T05:02:53.000+0000,6367521402730934,molabantisravanth@gmail.com,DELETE,"Map(predicate -> [""(emp_id#961 = 100)""])",null,List(508200138522909),0121-045452-7f69q673,21,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 209, numDeletedRows -> 0, scanTimeMs -> 208, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/12.2.x-scala2.12
21,2025-01-21T05:02:46.000+0000,6367521402730934,molabantisravanth@gmail.com,DELETE,"Map(predicate -> [""(emp_id#6511 = 100)""])",null,List(508200138522909),0121-045452-7f69q673,20,WriteSerializable,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 4563, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2075, numDeletedRows -> 3, scanTimeMs -> 1038, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1036)",null,Databricks-Runtime/12.2.x-scala2.12
20,2025-01-21T05:02:36.000+0000,6367521402730934,molabantisravanth@gmail.com,DELETE,"Map(predicate -> [""(emp_id#5576 = 200)""])",null,List(508200138522909),0121-045452-7f69q673,19,WriteSerializable,false,"Map(numRemovedFiles -> 4, numRemovedBytes -> 6080, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3758, numDeletedRows -> 4, scanTimeMs -> 2386, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1330)",null,Databricks-Runtime/12.2.x-scala2.12
19,2025-01-21T05:02:23.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,18,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
18,2025-01-21T05:02:19.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,17,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
17,2025-01-21T05:02:09.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,16,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1521)",null,Databricks-Runtime/12.2.x-scala2.12
16,2025-01-21T05:01:58.000+0000,6367521402730934,molabantisravanth@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(508200138522909),0121-045452-7f69q673,15,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1520)",null,Databricks-Runtime/12.2.x-scala2.12
15,2025-01-21T05:01:41.000+0000,6367521402730934,molabantisravanth@gmail.c

In [0]:
%sql

CREATE OR REPLACE TABLE scd2Demo(
  pk1 INT,
  pk2 STRING,
  dim1 INT,
  dim2 INT,
  dim3 INT,
  dim4 INT,
  active_status STRING,
  start_date TIMESTAMP,
  end_date TIMESTAMP
)
USING DELTA
LOCATION '/FileStore/tables/scd2Demo'

In [0]:
%sql

select * from scd2Demo

pk1,pk2,dim1,dim2,dim3,dim4,active_status,start_date,end_date


In [0]:
%sql

desc history scd2Demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2025-01-21T05:48:47.000+0000,6367521402730934,molabantisravanth@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(508200138522909),0121-045452-7f69q673,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12
